<a href="https://colab.research.google.com/github/Ximena5745/Desempeno_Institucional/blob/main/Consolidado_Ind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
import pandas as pd

In [2]:
!git clone https://github.com/Ximena5745/Desempeno_Institucional.git

Cloning into 'Desempeno_Institucional'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 1.19 MiB | 19.90 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
os.chdir("/content/Desempeno_Institucional/Data")

In [4]:
# 3️⃣ Buscar y leer todos los Excel
archivos_excel = glob.glob("*.xlsx") + glob.glob("*.xls")
df_todos = pd.concat(
    [pd.read_excel(archivo).assign(Archivo=archivo) for archivo in archivos_excel],
    ignore_index=True
)

In [5]:
# 4️⃣ Limpieza de datos
# Eliminar columnas específicas si existen
columnas_a_eliminar = ["dependencia", "incidencia", "objectivos_relacionados"]
df_todos = df_todos.drop(columns=[col for col in columnas_a_eliminar if col in df_todos.columns], errors="ignore")

# Eliminar filas donde 'fecha' esté vacía o nula
if "fecha" in df_todos.columns:
    df_todos = df_todos[df_todos["fecha"].notna() & (df_todos["fecha"].astype(str).str.strip() != "")]

# Reemplazar "Estrat&eacute;gico" por "Estratégico" en la columna 'clasificacion'
if "clasificacion" in df_todos.columns:
    df_todos["clasificacion"] = df_todos["clasificacion"].replace("Estrat&eacute;gico", "Estratégico")

# 5️⃣ Guardar el consolidado limpio
df_todos.to_excel("Consolidado.xlsx", index=False)

print("✅ Consolidado limpio guardado en:", os.path.join(os.getcwd(), "Consolidado.xlsx"))

✅ Consolidado limpio guardado en: /content/Desempeno_Institucional/Data/Consolidado.xlsx
